# Scheduling Agent with Agno(Phidata), Nebius AI and Cal.com

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1YbNqBKFE9BVSF0EHz2jVHB5x7y3pSS27?usp=sharing)

This notebook demonstrates how to build a scheduling assistant using Agno (PhiData) and Nebius AI Studio. The agent, powered by the meta-llama/Meta-Llama-3.1-70B-Instruct model, integrates with Cal.com to manage bookings, check available slots, and more.

Nebius AI Studio provides access to many state-of-the-art LLM models. Check out the full list of models here.

## Install Dependencies

In [ ]:
!pip install -qq agno python-dotenv requests pytz

## Setting Up Environment Variables

Visit https://studio.nebius.ai/ and sign up to get a Nebius API key. You’ll also need a Cal.com API key from https://cal.com/settings/developer/api-keys.

For security, we’ll prompt you to enter your Nebius and Cal.com API keys instead of hardcoding them, since Colab doesn’t natively support .env files.

In [ ]:
import os

os.environ["NEBIUS_API_KEY"] = "Your Nebius API Key"
os.environ["CALCOM_API_KEY"]="Your Cal.com API Key"
os.environ["CALCOM_EVENT_TYPE_ID"]="Your Cal.com Event Type"

In [ ]:
import os
import requests
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

api_key = os.environ["CALCOM_API_KEY"]
url = "https://api.cal.com/v2/event-types"
headers = {"Authorization": f"Bearer {api_key}"}

response = requests.get(url, headers=headers)

data = response.json()

event_types = data['data']['eventTypeGroups'][0]['eventTypes']

print("--- Event IDs ---")
for event in event_types:
    print(f"ID: {event['id']}, Title: {event['title']}")

print(f"\nTotal Event Types Found: {len(event_types)}")


--- Event IDs ---
ID: 870697, Title: 15 Min Meeting
ID: 870698, Title: 30 Min Meeting
ID: 870699, Title: Collaboration with Arindam

Total Event Types Found: 3


In [ ]:
import nest_asyncio

nest_asyncio.apply()

## Import Required Modules
We import essential modules for working with dates, Agno, and Cal.com tools.

In [ ]:
from datetime import datetime, timedelta
from agno.agent import Agent
from agno.models.nebius import Nebius
from agno.tools.calcom import CalComTools


## Define Instructions for the Agent

Set up the instructions that guide the agent’s scheduling capabilities.

In [ ]:
# Define instructions with current date
INSTRUCTIONS = f"""You're a scheduling assistant. Today is {datetime.now()}.
You can help users by:
    - Finding available time slots using get_available_slots(start_date, end_date)
    - Creating new bookings using create_booking(start_time, name, email)
    - Managing existing bookings using get_upcoming_bookings(email)
    - Rescheduling bookings using reschedule_booking(booking_uid, new_start_time, reason)
    - Cancelling bookings using cancel_booking(booking_uid, reason)

IMPORTANT STEPS for booking:
1. First check available slots using get_available_slots
2. Then create booking using create_booking with the exact start_time, name, and email provided
3. Finally verify the booking was created using get_upcoming_bookings with the provided email

When asked to book a call, you MUST:
1. Call create_booking with the provided start_time, name, and email
2. Then verify the booking using get_upcoming_bookings
3. Confirm to the user whether the booking was successful

Remember:
- Dates should be in YYYY-MM-DD format
- Times should be in YYYY-MM-DDTHH:MM:SS+TZ format (e.g., 2025-03-20T14:30:00+05:30)
- Always confirm details before making changes
"""

## Initialize the Cal.com Tools and Model

Set up the Cal.com tools and Nebius AI model for the agent.

In [ ]:
# Create the CalCom tools instance
try:
    calcom_tools = CalComTools(
        user_timezone="Asia/Kolkata",
        api_key=os.environ["CALCOM_API_KEY"],
        event_type_id=os.environ["CALCOM_EVENT_TYPE_ID"]
    )
except Exception as e:
    print(f"Error initializing CalCom tools: {e}")
    raise

## Create the Scheduling Agent
Instantiate the agent with the defined instructions, model, and tools.

In [ ]:
# Create the agent
agent = Agent(
    name="Calendar Assistant",
    instructions=[INSTRUCTIONS],
    model=Nebius(
        id="Qwen/Qwen3-30B-A3B",
        api_key=os.getenv("NEBIUS_API_KEY") # Explicitly pass the API key
    ),
    tools=[calcom_tools],
    show_tool_calls=True,
    tool_choice="auto",
    markdown=True,
)

## Run an Example Booking
Test the agent by checking available slots and booking a call.

In [ ]:
# Example function to book a call
def book_example_call():
    # Get today's date and tomorrow's date
    today = datetime.now().strftime("%Y-%m-%d")
    tomorrow = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")

    # Check available slots
    print("Checking available slots...")
    agent.print_response(f"""
    Please check available slots between {today} and {tomorrow}
    """)

    # Book a specific slot
    print("\nAttempting to book a call...")
    agent.print_response("""
    Please book a call with these details:
    - Start Time: 2025-03-22T22:30:00+05:30
    - Name: Agno Agent Test
    - Email: studioone.tech@gmail.com

    After booking, please verify the booking exists.
    """)

# Run the example
book_example_call()